In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import scipy.stats as stats

/Users/reef/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/reef/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
senate = pd.read_csv('./data/senate.csv')

In [3]:
senate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219539 entries, 0 to 219538
Data columns (total 10 columns):
created_at       219539 non-null object
id_str           219539 non-null float64
reply_count      219539 non-null float64
retweet_count    219539 non-null float64
text             219539 non-null object
user             219539 non-null object
name             219539 non-null object
state            219539 non-null object
party            219539 non-null object
the_ratio        219539 non-null float64
dtypes: float64(4), object(6)
memory usage: 16.7+ MB


In [4]:
senate.head()

,created_at,id_str,reply_count,retweet_count,text,user,name,state,party,the_ratio
0,Sun May 21 19:26:26 +0000 2017,8.660000e+17,116.0,174.0,"Franni here. Since it's Al's birthday, and sin...",alfranken,Al Franken,New York,Democratic,0.666667
1,Tue May 16 01:44:44 +0000 2017,8.640000e+17,718.0,1248.0,This is profoundly troubling. Why would Presid...,alfranken,Al Franken,New York,Democratic,0.575321
2,Wed May 10 19:55:37 +0000 2017,8.620000e+17,218.0,1334.0,It couldn't be clearer: we need an independent...,alfranken,Al Franken,New York,Democratic,0.163418
3,Wed May 10 19:54:55 +0000 2017,8.620000e+17,134.0,794.0,More troubling news: AG Sessions was involved ...,alfranken,Al Franken,New York,Democratic,0.168766
4,Wed May 10 19:54:20 +0000 2017,8.620000e+17,131.0,556.0,Troubling news that you probably know by now: ...,alfranken,Al Franken,New York,Democratic,0.235612


In [23]:
senate.the_ratio.describe()

count    219539.000000
mean          0.677336
std           1.847231
min           0.000000
25%           0.046512
50%           0.250000
75%           0.636364
max         138.000000
Name: the_ratio, dtype: float64

In [32]:
np.percentile(senate.the_ratio, 25)

0.04651162790697674

In [51]:
for ix, each in enumerate(senate['the_ratio'][:5]):
    print(ix, each <= np.percentile(senate.the_ratio, 25))

0 False
1 False
2 False
3 False
4 False


In [56]:
temporary_list = []

for ix, each in enumerate(['dog', 'cat']):
    print(ix, each)
    temporary_list.append(f'The index is: {ix}')

0 dog
1 cat


In [57]:
temporary_list

['The index is: 0', 'The index is: 1']

In [ ]:
new_array_to_make_into_column = []

for each:
    if condition:
        new_array_to_make_into_column.append(value)
    elif other_condition:
        new_array_to_make_into_column.append(other_value)
        
the_df['new_column'] = new_array_to_make_into_column

In [58]:
# engineering quartiles for the ratio classification


for ix, each in enumerate(senate['the_ratio']):
    if each <= np.percentile(senate.the_ratio, 25):
        senate['ratio_quartile'][ix] = 1
    elif each <= np.percentile(senate.the_ratio, 50):
        senate['ratio_quartile'][ix] = 2
    elif each <= np.percentile(senate.the_ratio, 75):
        senate['ratio_quartile'][ix] = 3
    elif each <= np.percentile(senate.the_ratio, 100):
        senate['ratio_quartile'][ix] = 4

ValueError: Cannot setitem on a Categorical with a new category, set the categories first

In [36]:
senate.ratio_quartile.value_counts()

1    219539
Name: ratio_quartile, dtype: int64

In [ ]:
y = senate['ratio_quartile']
X = senate['text']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=42)

le = LabelEncoder()
le.fit(y)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [ ]:
%%time

#with count vectorizer
pipeline = Pipeline([
    ('vect', CountVectorizer()), 
    ('rfc', RandomForestClassifier())
]) 
pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
print(pipeline.score(X_test, y_test))